In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
taxi=sc.textFile("yellow.csv")
bike = sc.textFile("citibike.csv")

In [21]:
def filterBike(pid, records):
    if pid==0:
        next(records)
    for record in records:
        fields = record.split(',')
        if (fields[6]=='Greenwich Ave & 8 Ave' and
            fields[3].startswith('2015-02-01')):
            yield (fields[3][:19], 1)
            
matchedBike = bike.mapPartitionsWithIndex(filterBike)                        

In [22]:
matchedBike.take(2)

[('2015-02-01 00:05:00', 1), ('2015-02-01 00:05:00', 1)]

In [23]:
station = (-74.00263761,40.7390169)

In [29]:
import os
os.environ["PROJ_LIB"] = 'C:\\Users\\sonali\\Anaconda3\\Library\\share'

In [30]:
import pyproj
pyproj.__file__

'C:\\Users\\sonali\\Anaconda3\\lib\\site-packages\\pyproj\\__init__.py'

In [32]:
def filterTaxi(pid, lines):
    import os
    os.environ["PROJ_LIB"] = 'C:\\Users\\sonali\\Anaconda3\\Library\\share'
    if pid==0:
        next(lines)
    import pyproj
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
    station = proj(-74.00263761, 40.7390169)
    radius = 1320**2
    for trip in lines:
        fields=trip.split(',')
        if 'NULL' in fields[4:6]:continue
        dropoff = proj(fields[5],fields[4])
        distance = ((station[0]-dropoff[0]**2+station[1]-dropoff[1]**2))
        if distance<=radius:
            yield(fields[1][:19],0)
        
matchedTaxi = taxi.mapPartitionsWithIndex(filterTaxi)
#matchedTaxi.count()
matchedTaxi.take(2)

[('2015-02-01 00:10:10', 0), ('2015-02-01 00:03:36', 0)]

In [33]:
def filterTaxi(pid, lines):
    import os
    os.environ["PROJ_LIB"] = 'C:\\Users\\sonali\\Anaconda3\\Library\\share'
    if pid==0:
        next(lines)
    import pyproj
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
    station = proj(-74.00263761, 40.7390169)
    radius = 1320**2
    for trip in lines:
        fields = trip.split(',')
        if 'NULL' in fields[4:6]: continue
        dropoff = proj(fields[5], fields[4])
        distance = ((station[0]-dropoff[0])**2 +
                   (station[1]-dropoff[1])**2)
        if distance <= radius:
            yield(fields[1][:19], 0)

matchedTaxi = taxi.mapPartitionsWithIndex(filterTaxi)
matchedTaxi.take(2)

[('2015-02-01 00:11:03', 0), ('2015-02-01 00:10:23', 0)]

In [34]:
alltrips = (matchedBike+matchedTaxi).sortByKey().cache()

In [37]:
def connectTrips(_, records):
    import datetime
    lastTaxiTime = None
    count = 0
    for dt, mode in records:
        t = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        if mode==1:
            if lastTaxiTime!=None:
                diff = (t-lastTaxiTime).total_seconds()
                if diff>=0 and diff<=10:
                    count += 1
        else:
            lastTaxiTime = t
    yield count
    
count = alltrips.mapPartitionsWithIndex(connectTrips) \
    .reduce(lambda x,y:x+y) 
#sc.parallelize([count],1).saveAsTextFile('tmp.txt')

print(count)

42
